In [1]:
import os
os.chdir('/home')

import numpy as np
import pandas as pd
import gym

from pavlov import pipeline
from pavlov.pipeline import transformations
from pavlov import models
from pavlov import actors
from pavlov import agents
from pavlov import auxiliary

from keras import optimizers

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v1')
base_config = {
    'layer_sizes': [128],
    'activation': 'relu'
}
dqn_config = {
    'gamma': 0.99,
    'tau': 1.0,
    'optimizer': optimizers.SGD(lr=0.03)
}
topology = models.topology.DenseTopology(**base_config)
model = models.DQNModel(topology, **dqn_config)

epsilon_schedule = auxiliary.schedules.LinearDecay(1.0, 0.1, 500, -1)
actor = actors.EpsilonGreedyActor(epsilon_schedule)
buffer_size = 10000
batch_size = 64

pline = pipeline.Pipeline()
agent = agents.Agent(env,
                     state_pipeline=pline,
                     model=model, actor=actor,
                     buffer_size=buffer_size, batch_size=batch_size,
                     report_freq=10, warmup_length=50)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
agent.run_indefinitely()

Terminating.


/home/pavlov/models/dqn.py:76: RuntimeWarning: invalid value encountered in multiply
  targets = targets * actions  # makes it one-hot, value in the place of the action


ValueError: Model has hit NaN weights: check #37 here: https://tinyurl.com/yahuwbno

In [16]:
any(np.isnan(w).any() for w in model.model.get_weights())

True

In [ ]:
# CARTPOLE

env = gym.make('Breakout-v0')

topology_config = {
    'layer_sizes': [128],
    'activation': 'relu'
}
topology = models.topology.DenseTopology(**topology_config)

dqn_config = {
    'gamma': 0.99,
    'tau': 1.0,
    'optimizer': optimizers.Adam(0.0001)
}
model = models.DQNModel(topology, **dqn_config)

epsilon_schedule = auxiliary.schedules.LinearDecay(1.0, 0.1, 500, -1)
actor = actors.EpsilonGreedyActor(epsilon_schedule)
buffer_size = 10000
batch_size = 64

pline = pipeline.Pipeline()
pline.add(transformations.rgb_to_grey())
pline.add(transformations.downsample(new_shape=(84, 84)))
pline.add(transformations.combine_consecutive(2, 'max'))
pline.add(transformations.stack_consecutive(4))

agent = agents.Agent(env,
                     state_pipeline=pline,
                     model=model, actor=actor,
                     buffer_size=buffer_size, batch_size=batch_size,
                     report_freq=1, warmup_length=5)
agent.run_episode(render=True)

In [ ]:
agent.episode_to_mp4(1, '/home/videos')